In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy.stats as stats


In [8]:
df=pd.read_pickle('ml_ready_ultimate_data_challenge.pkl')
df.head()

,avg_dist,avg_rating_by_driver,avg_rating_of_driver,avg_surge,surge_pct,ultimate_black_user,trips_in_first_30_days,weekday_pct,city_Astapor,city_King's Landing,...,phone_Computer,phone_iPhone,signup_weekday_Fri,signup_weekday_Mon,signup_weekday_Sat,signup_weekday_Sun,signup_weekday_Thurs,signup_weekday_Tues,signup_weekday_Wed,retained
0,3.67,5.0,4.7,1.10,15.4,1,4,46.2,0,1,...,0,1,0,0,1,0,0,0,0,1
1,8.26,5.0,5.0,1.00,0.0,0,0,50.0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0.77,5.0,4.3,1.00,0.0,0,3,100.0,1,0,...,0,1,0,1,0,0,0,0,0,0
3,2.36,4.9,4.6,1.14,20.0,1,9,80.0,0,1,...,0,1,1,0,0,0,0,0,0,1
4,3.13,4.9,4.4,1.19,11.8,0,14,82.4,0,0,...,0,0,0,1,0,0,0,0,0,0


In [17]:
from sklearn.model_selection import train_test_split
X=df.loc[:,'avg_dist':'signup_weekday_Wed'].values

y= df.retained.values
X_train, X_test,y_train, y_test=train_test_split(X,y, test_size=.2, random_state=40)

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [25]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
lr=LogisticRegression(random_state = 40)
c = {
 'C':[0.001, 0.01, 0.1, 1, 10]}
gsearch_lr = GridSearchCV(estimator = lr, 
 param_grid = c, scoring='roc_auc', cv=5)
gsearch_lr.fit(X_train_scaled, y_train)

print('the optimal value for c is: {} with a score of: {}'.format(gsearch_lr.best_params_, gsearch_lr.best_score_))

the optimal value for c is: {'C': 0.1} with a score of: 0.757101237566


In [26]:
lr = LogisticRegression(C=0.1,random_state = 40)
lr.fit(X_train_scaled, y_train)
y_train_pred_lr = lr.predict(X_train_scaled)
y_test_pred_lr = lr.predict(X_test_scaled)
train_predprob_lr = lr.predict_proba(X_train_scaled)[:,1]
test_predprob_lr = lr.predict_proba(X_test_scaled)[:,1]

In [38]:
from sklearn.metrics import roc_auc_score,accuracy_score, confusion_matrix

print("Training scores of logistic regression: {}".format(roc_auc_score(y_train, train_predprob_lr)))
print("Training accuracy of logistic regression: {}".format(accuracy_score(y_train, y_train_pred_lr)))
print("Confusion matrix of logistic regression: {}".format(confusion_matrix(y_train, y_train_pred_lr)))

Training scores of logistic regression: 0.75780815733
Training accuracy of logistic regression: 0.704686330289
Confusion matrix of logistic regression: [[16017  3574]
 [ 6288  7516]]


In [39]:
print("Test set scores of logistic regression: {}".format(roc_auc_score(y_test, test_predprob_lr)))
print("Training accuracy of logistic regression: {}".format(accuracy_score(y_test, y_test_pred_lr)))
print("Confusion matrix of logistic regression: {}".format(confusion_matrix(y_test, y_test_pred_lr)))

Test set scores of logistic regression: 0.764164134319
Training accuracy of logistic regression: 0.70918672895
Confusion matrix of logistic regression: [[4067  928]
 [1500 1854]]


In [42]:
df_coeffs = pd.DataFrame(zip(df.loc[:,'avg_dist':'signup_weekday_Wed'], lr.coef_.flatten())).sort_values(by=[1], ascending=False)
df_coeffs

,0,1
5,ultimate_black_user,0.437841
9,city_King's Landing,0.428185
6,trips_in_first_30_days,0.419748
13,phone_iPhone,0.245886
4,surge_pct,0.074873
17,signup_weekday_Sun,0.032606
16,signup_weekday_Sat,0.024732
7,weekday_pct,0.008190
12,phone_Computer,0.005767
15,signup_weekday_Mon,0.002010


2. The model I used was logistic regression.I chose this model because it it nearly complete right out of the box and I prioritsed speed in this situation. While the model only has 70% accuracy it is useful because it can show us which users may be at higher rsk of not being retained. 

3. This model has two uses. Firstly it can be used to identify users who may be at risk to defect. Secondly, using the coeffincient scores we can see factors that affect a riders likelihood to continue using the platform. One key insight from the coefficients is that Ultimate should offer more discounts for their black cars because users who ride in a black car are far more likely to keep using the platform. Additionally, it appears our android app may have some problems because android users are far less likely to retain. Furthermore, we may want to do a deeper dive on users from Astapor because using the app from their is our leading negitive indicator. 